In [1]:
import requests, json
from lxml import html
from bs4 import BeautifulSoup
import time

In [2]:
url = "https://data.cdc.gov/browse?category=Vaccinations"

In [3]:
response1 = requests.get(url)
soup = BeautifulSoup(response1.text, 'html.parser')
#print(soup)
datasets = []
for link in soup.find_all('a'):
  path = link.get('href')
  try:
    if "http" in path:
      datasets.append(path)
  except:
    pass
print(datasets)

['http://www.cdc.gov/', 'http://dev.socrata.com/', 'http://www.facebook.com/CDC', 'https://github.com/CDCgov', 'http://twitter.com/CDCgov', 'http://www.youtube.com/CDCstreamingHealth', 'https://instagram.com/CDCgov/', 'http://dev.socrata.com/', 'http://www.facebook.com/CDC', 'https://github.com/CDCgov', 'http://twitter.com/CDCgov', 'http://www.youtube.com/CDCstreamingHealth', 'https://instagram.com/CDCgov/', 'https://data.cdc.gov/Pregnancy-Vaccination/Map-of-Seasonal-Vaccination-Coverage-with-live-bir/rxbz-avfv', 'http://www.cdc.gov/prams/', 'http://www.cdc.gov/mmwr/preview/mmwrhtml/mm6107a1.htm?s_cid=mm6107a1_e', 'https://data.cdc.gov/Vaccinations/COVID-19-Vaccine-Distribution-Allocations-by-Juris/w9zu-fywh', 'https://www.hhs.gov/coronavirus/covid-19-vaccines/index.html', 'https://data.cdc.gov/Vaccinations/COVID-19-Vaccine-Initial-Allocations-Pfizer/saz5-9hgg', 'https://data.cdc.gov/Vaccinations/COVID-19-Vaccine-Distribution-Allocations-by-Juris/b7pe-5nws', 'https://dev.socrata.com/fo

In [4]:
dataset_url = datasets[-21]
#data_url = url[:len(url) - 9] + dataset_url
print(dataset_url)

https://dev.socrata.com/foundry/data.cdc.gov/rh2h-3yt2


In [5]:
# print(type(dataset_url))
response2 = requests.get("https://data.cdc.gov/resource/8xkx-amqh.json")


In [6]:
json_object = json.loads(response2.text)
print(json.dumps(json_object, indent = 1))

[
 {
  "date": "2022-04-26T00:00:00.000",
  "fips": "05067",
  "mmwr_week": "17",
  "recip_county": "Jackson County",
  "recip_state": "AR",
  "completeness_pct": "90.5",
  "administered_dose1_recip": "7638",
  "administered_dose1_pop_pct": "45.7",
  "administered_dose1_recip_5plus": "7636",
  "administered_dose1_recip_5pluspop_pct": "48.4",
  "administered_dose1_recip_12plus": "7479",
  "administered_dose1_recip_12pluspop_pct": "51.6",
  "administered_dose1_recip_18plus": "7050",
  "administered_dose1_recip_18pluspop_pct": "52.8",
  "administered_dose1_recip_65plus": "2343",
  "administered_dose1_recip_65pluspop_pct": "76.7",
  "series_complete_yes": "6528",
  "series_complete_pop_pct": "39",
  "series_complete_5plus": "6526",
  "series_complete_5pluspop_pct": "41.4",
  "series_complete_5to17": "415",
  "series_complete_5to17pop_pct": "17.2",
  "series_complete_12plus": "6424",
  "series_complete_12pluspop_pct": "44.3",
  "series_complete_18plus": "6111",
  "series_complete_18pluspop_

In [7]:
import numpy as np
import pandas as pd
import os

In [9]:
area = input("Select Washington county or Michigan State")
if area == "Washington County":
  df_Washington = pd.read_json("https://data.cdc.gov/resource/8xkx-amqh.json")
  df_Washington = df_Washington.loc[df_Washington['recip_county']=='Washington County']
  df_Washington.to_csv("WashingtonCounty.csv", index=False)
  print(df_Washington)
elif area == "Michigan State":
  df_Michigan = pd.read_csv(r"COVID-19.csv")
  df_Michigan = df_Michigan.loc[df_Michigan['Recip_State'] == 'MI']
  df_Michigan.to_csv('MIState.csv', index=False)
  print(df_Michigan)

Select Washington county or Michigan StateMichigan State
               Date   FIPS  MMWR_week        Recip_County Recip_State  \
74       04/25/2022  26153         17  Schoolcraft County          MI   
126      04/25/2022  26011         17       Arenac County          MI   
127      04/25/2022  26057         17      Gratiot County          MI   
128      04/25/2022  26081         17         Kent County          MI   
129      04/25/2022  26137         17       Otsego County          MI   
...             ...    ...        ...                 ...         ...   
1637538  12/13/2020  26157         51      Tuscola County          MI   
1637599  12/13/2020  26033         51     Chippewa County          MI   
1637604  12/13/2020  26015         51        Barry County          MI   
1637606  12/13/2020  26135         51       Oscoda County          MI   
1637633  12/13/2020  26001         51       Alcona County          MI   

         Completeness_pct  Administered_Dose1_Recip  \
74         

In [10]:
import csv
import sqlite3
DB_NAME = "Covid_Data.sqlite3"

In [11]:
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    drop_covid_db_sql = "DROP TABLE IF EXISTS 'County_Covid_Data'"

    create_covid_db_sql = '''
        CREATE TABLE IF NOT EXISTS "County_Covid_Data" (
            "Id" INTEGER PRIMARY KEY AUTOINCREMENT,
            "Date" DATETIME NOT NULL,
            "Recip_County" TEXT NOT NULL,
            "Recip_State" TEXT NOT NULL,
            "Completeness_pct" FLOAT NOT NULL,
            "Booster_Doses" FLOAT NOT NULL,
            "SVI_CTGY" TEXT NOT NULL,
            "Metro_status" TEXT NOT NULL
        )
    '''
    cur.execute(drop_covid_db_sql)
    cur.execute(create_covid_db_sql)

    conn.commit()
    conn.close()


In [12]:
def populate_database():

    data_header = []
    data_rows = []

    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    with open("COVID-19.csv", 'r') as csvfile:
        data = []
        csv_header = csv.reader(csvfile)
        for h in csv_header:
            data.append(h)
        data_header.extend(data[0])
        data_rows.extend(data[1:])

    insert_covid_db_sql = '''
        INSERT INTO County_Covid_Data
        VALUES (NULL, ?,?,?,?,?,?,?)
    '''

    for dr in data_rows:
        cur.execute(insert_covid_db_sql, [
            dr[0], dr[3], dr[4], dr[5], dr[28], dr[38], dr[-21]
        ])

    conn.commit()
    conn.close()


In [13]:
create_database()
populate_database()

In [14]:
conn = sqlite3.connect(DB_NAME)
cur = conn.cursor()
calling_command = '''
        SELECT * FROM County_Covid_Data LIMIT 20
    '''

print(cur.execute(calling_command).fetchall())

[(1, '04/25/2022', 'Bullitt County', 'KY', 94.0, 15870.0, 'A', 'Metro'), (2, '04/25/2022', 'Lawrence County', 'KY', 94.0, 3006.0, 'D', 'Non-metro'), (3, '04/25/2022', 'Todd County', 'KY', 94.0, 1909.0, 'D', 'Non-metro'), (4, '04/25/2022', 'Assumption Parish', 'LA', 96.9, 3549.0, 'D', 'Non-metro'), (5, '04/25/2022', 'Howard County', 'MD', 98.2, 166742.0, 'A', 'Metro'), (6, '04/25/2022', 'Cass County', 'MN', 94.6, 9830.0, 'B', 'Non-metro'), (7, '04/25/2022', 'Stearns County', 'MN', 94.6, 47744.0, 'B', 'Metro'), (8, '04/25/2022', 'Jefferson County', 'MO', 91.3, 46776.0, 'A', 'Metro'), (9, '04/25/2022', 'Hinds County', 'MS', 98.1, 56046.0, 'D', 'Metro'), (10, '04/25/2022', 'Monroe County', 'MS', 98.1, 6814.0, 'C', 'Non-metro'), (11, '04/25/2022', 'Hill County', 'MT', 95.8, 4735.0, 'C', 'Non-metro'), (12, '04/25/2022', 'Roosevelt County', 'MT', 95.8, 3393.0, 'D', 'Non-metro'), (13, '04/25/2022', 'Duplin County', 'NC', 96.9, 6026.0, 'D', 'Non-metro'), (14, '04/25/2022', 'Grant County', 'ND',

In [15]:
def access_db(state, svi, m):
  conn = sqlite3.connect(DB_NAME)
  cur = conn.cursor()
  fetch_svi = f'''SELECT Completeness_pct, Recip_County FROM County_Covid_Data WHERE Recip_State == '{state}' AND SVI_CTGY == '{svi}' AND Metro_status == '{m}' LIMIT 20 '''
  return cur.execute(fetch_svi).fetchall()
  # return fetch_svi
  conn.commit()
  conn.close()




In [16]:
# def county_db_access(s, svi):
#   conn = sqlite3.connect(DB_NAME)
#   cur = conn.cursor()
#   fetch_svi = f'''SELECT UNIQUE Recip_County FROM County_Covid_Data WHERE Recip_State == {s} AND SVI_CTGY == {svi} LIMIT 20 '''
#   print(cur.execute(fetch_svi).fetchall())
#   conn.commit()
#   conn.close()

In [17]:
# conn = sqlite3.connect(DB_NAME)
# cur = conn.cursor()
# test = '''
# SELECT * FROM County_Covid_Data WHERE Recip_State == 'NY' LIMIT 20
# '''
# test_mi = cur.execute(test).fetchall()
# print(test_mi)


In [18]:
states_list = ['MI', 'NY', 'UT', 'OH', 'NC', 'GA', 'IL', 'KY', 'MS', 'MO']
metro_list = ['Metro', 'Non-metro']
SVI_Ctgy = ['A', 'B', 'C', 'D']
tree = {}

def create_tree():
  for s in states_list:
    tree[s] = {}
    for m in metro_list:
      tree[s][m] = {}
      for svi in SVI_Ctgy:
        fetch_vaccine = access_db(s, svi, m)
        #print(fetch_vaccine)
        tree[s][m][svi] = fetch_vaccine
        # county_names = county_db_access(s,svi)
  return tree

In [ ]:
create_tree()

In [ ]:
import pprint
pprint.pprint(create_tree())